In [1]:

from pypower.api import loadcase, runopf, ppoption
from pypower.idx_brch import *
from pypower.idx_bus import *
from pypower.idx_gen import *
from pypower.idx_cost import *
from pypower.ext2int import ext2int
import oct2py
import numpy as np
from numpy import r_
import math
import os, sys
from tqdm import tqdm
from oct2py import Oct2Py
octave = Oct2Py()
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pickle
import shutil
import cvxpy as cp
from contextlib import contextmanager
import warnings
import cyipopt
import json
from problemDefJITP import opfSocp
from copy import deepcopy
# suppress ComplexWarning
warnings.simplefilter("ignore", np.ComplexWarning)
# check for latex and configure matplotlib accordingly
if shutil.which('latex') is None:
    LATEX_AVAILABLE = False
else:
    LATEX_AVAILABLE = True
if LATEX_AVAILABLE:
    plt.rcParams['font.size'] = 14
    plt.rcParams['text.usetex'] = True
    plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,amssymb}'

# to suppress output from functions    
@contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout    

# user options 
MAX_BUS = 10000 # upper limit of number of buses in cases to be considered
RATE = 'A' # which line rating to use ('A','B','C')

In [8]:
# get all cases in current directory
current_directory = os.getcwd()+'/'
# current_directory = '/home/sbose/pglib-opf/' # for running on BEBOP
all_files_and_directories = os.listdir(current_directory)
case_files = [os.path.join(current_directory, f) for f in all_files_and_directories if f.endswith('.m') and os.path.isfile(os.path.join(current_directory, f))]

cases, casenames = [], []
cases_ext = []
cases_full, casenames_full = [], []
for cf in (t:=tqdm(case_files)):
    try:
        octave.source(current_directory+os.path.basename(cf))
    except:
        continue
    cname = os.path.basename(cf).split('.')[0]
    num_buses = None
    # determine number of buses in the case from its name
    for ci in cname.split('_'):
        if 'case' in ci:
            num_buses = int(''.join([chr for chr in ci.replace('case','',1) if chr.isdigit()]))
    
    # fitler out cases with more buses than MAX_BUS
    if num_buses <= MAX_BUS:
        # convert to internal indexing
        case_orig = loadcase(octave.feval(cname))
        case_correct_idx = ext2int(case_orig)
        # check for repeated branches
        buslist = [(f,t) for f,t in zip(case_correct_idx['branch'][:,F_BUS],case_correct_idx['branch'][:,T_BUS])]
        if len(buslist) != len(set(buslist)):
            print(f"{len(buslist)-len(set(buslist))} non unique branches detected for {cname}!")
        else:
            print(f"{cname} has all unique branches.")
        # append
        cases_ext.append(cases)
        cases.append(case_correct_idx)
        casenames.append(cname)
    t.set_description(cname)

pglib_opf_case60_c:   0%|                                                            | 0/58 [00:00<?, ?it/s]

16 non unique branches detected for pglib_opf_case60_c!


pglib_opf_case3120sp_k:   3%|█▋                                              | 2/58 [00:00<00:06,  8.28it/s]

9 non unique branches detected for pglib_opf_case3120sp_k!


pglib_opf_case39_epri:   5%|██▌                                              | 3/58 [00:00<00:09,  6.06it/s]

...

934 non unique branches detected for pglib_opf_case6468_rte!
223 non unique branches detected for pglib_opf_case1888_rte!


pglib_opf_case1803_snem:  98%|█████████████████████████████████████████████▏| 57/58 [00:11<00:00,  5.38it/s]

281 non unique branches detected for pglib_opf_case1354_pegase!
583 non unique branches detected for pglib_opf_case1803_snem!


pglib_opf_case793_goc: 100%|████████████████████████████████████████████████| 58/58 [00:11<00:00,  5.00it/s]

9 non unique branches detected for pglib_opf_case793_goc!


In [3]:
# tailoring to only three cases
idx1 = casenames.index('pglib_opf_case2312_goc')
# idx2 = casenames.index('pglib_opf_case4661_sdet')
# idx3 = casenames.index('pglib_opf_case10000_goc')
casenames = [casenames[cidx] for cidx in [idx1]]
cases = [cases[cidx] for cidx in [idx1]]

In [4]:
for cn,c,ci in tqdm(zip(casenames,cases,range(len(cases)))):
    obj = opfSocp(c)
    cub, clb = obj.calc_cons_bounds()
    xub, xlb = obj.calc_var_bounds()
    prob = cyipopt.Problem(
        n = obj.in_size,
        m = obj.cons_size,
        problem_obj=obj,
        lb=xlb,
        ub=xub,
        cl=clb,
        cu=cub
    )
    prob.add_option('tol',1e-6)
    prob.add_option('max_iter',2500)
    prob.add_option('mumps_mem_percent',25000)
    prob.add_option('mu_max',1e-1)
    prob.add_option('mu_init',1e-1)
    x, info = prob.solve(obj.calc_x0_flatstart())

0it [00:00, ?it/s]

RepeatedSet: Variable ('rSf', 8623) already set!
RepeatedSet: Constraint ('balance_real', 245) already set!


IndexError: obus is [249, 250, 250, 255, 261, 222]. Error!

In [ ]:
# # compare all methods
# results = [{'jsoc_va':[],'jsoc_sf':[],'jsoc_st':[],'jsoc_pg':[],'jsoc_qg':[],
#             'jnl_va':[],'jnl_sf':[],'jnl_st':[],'jnl_pg':[],'jnl_qg':[],
#             'soc_va':[],'soc_sf':[],'soc_st':[],'soc_pg':[],'soc_qg':[],
#             'nl_va':[],'nl_sf':[],'nl_st':[],'nl_pg':[],'nl_qg':[],
#             } for _ in casenames]

# # make figure
# fig, axs = plt.subplots(4, len(casenames), figsize=(6*4, 5*len(casenames)))

# # delete temp.m if it already exists
# if os.path.exists('temp.m'):
#     os.remove('temp.m')

# for cn,c,ci in tqdm(zip(casenames,cases,range(len(cases)))):
#     # extract SOC solution and exclude cases for which it did not solve exactly
#     with open(os.getcwd()+'/'+cn+'_socp_solved.pkl','rb') as file:
#         dct_socp = pickle.load(file)
#     SOCP_SOLVED = True if dct_socp['info'] == 0 else False # whether case was solved to local minima
#     socpObj = opfSocp(c)
#     U_socp, W_socp, Sg_socp, Sf_socp, St_socp = dct_socp['U'], dct_socp['W'], dct_socp['Sg'], dct_socp['Sf'], dct_socp['St']
#     # extract vmag, Sf, St, Pg, Qg, fmax from the loaded case
#     vm_socp = np.sqrt(U_socp)
#     Pg_socp, Qg_socp = Sg_socp.real, Sg_socp.imag
#     fmax_socp = socpObj.flow_lim
#     # extract MATPOWER nl solution 
#     with open(os.getcwd()+'/'+cn+'_solved.pkl','rb') as file:
#         dct_nl = pickle.load(file)
#     dct_nl = ext2int(dct_nl) # convert into internal indexing
#     # extract vmag, Sf, St, Pg, Qg, fmax from the loaded case
#     vm_nl = dct_nl['bus'][:,VM]
#     Sf_nl = (dct_nl['branch'][:,PF] + 1j * dct_nl['branch'][:,QF]) / dct_nl['baseMVA']
#     St_nl = (dct_nl['branch'][:,PT] + 1j * dct_nl['branch'][:,QT]) / dct_nl['baseMVA']
#     Pg_nl, Qg_nl = dct_nl['gen'][:,PG] / dct_nl['baseMVA'], dct_nl['gen'][:,QG] / dct_nl['baseMVA']
#     fmax_nl = dct_nl['branch'][:,RATE_A] / dct_nl['baseMVA']
#     # extract Julia socp solution
#     with open(os.getcwd()+'/'+cn+'_julia_soc_solved.json','rb') as file:
#         dct_jsocp = json.load(file)
#     # dump output to text file
#     with open('temp.m','w') as file:
#         file.write(dct_jsocp)
#     dct_jsocp = ext2int(octave.feval('temp.m')) # convert into internal indexing
#     os.remove('temp.m') # delete temporary .m file
#     vm_jsocp = dct_jsocp['bus'][:,VM]
#     Sf_jsocp = (dct_jsocp['branch'][:,PF] + 1j * dct_jsocp['branch'][:,QF]) / dct_jsocp['baseMVA']
#     St_jsocp = (dct_jsocp['branch'][:,PT] + 1j * dct_jsocp['branch'][:,QT]) / dct_jsocp['baseMVA']
#     Pg_jscop, Qg_jsocp = dct_jsocp['gen'][:,PG] / dct_jsocp['baseMVA'], dct_jsocp['gen'][:,QG] / dct_jsocp['baseMVA']
#     fmax_jsocp = dct_jsocp['branch'][:,RATE_A] / dct_jsocp['baseMVA']
#     # extract Julia nl solution
#     with open(os.getcwd()+'/'+cn+'_julia_nl_solved.json','rb') as file:
#         dct_jnl = json.load(file)
#     # dump output to text file
#     with open('temp.m','w') as file:
#         file.write(dct_jnl)
#     dct_jnl = ext2int(octave.feval('temp.m')) # convert into internal indexing
#     os.remove('temp.m') # delete temporary .m file
#     vm_jnl = dct_jnl['bus'][:,VM]
#     Sf_jnl = (dct_jnl['branch'][:,PF] + 1j * dct_jnl['branch'][:,QF]) / dct_jnl['baseMVA']
#     St_jnl = (dct_jnl['branch'][:,PT] + 1j * dct_jnl['branch'][:,QT]) / dct_jnl['baseMVA']
#     Pg_jnl, Qg_jnl = dct_jnl['gen'][:,PG] / dct_jnl['baseMVA'], dct_jnl['gen'][:,QG] / dct_jnl['baseMVA']
#     fmax_jnl = dct_jnl['branch'][:,RATE_A] / dct_jnl['baseMVA']

In [ ]:
101492 - 100760

732

In [ ]:
np.repeat([1,2,3],[1,1,3])

array([1, 2, 3, 3, 3])

In [ ]:
np.repeat([1,1,2],[0,1,2])

array([1, 2, 2])

In [ ]:
obj.n_gen

226

In [ ]:
obj.num_indices_per_constr.size

31741

In [ ]:
np.repeat(obj.active_constr,obj.num_indices_per_constr).shape

(101492,)

In [ ]:
obj.jac_const.size

100760

In [ ]:
len(obj.jac_coeff)

100760